# **<center>Preprocessing**

In [1]:
import os
from dotenv import load_dotenv

import pandas as pd
import numpy as np

import datetime as dt
from tqdm import tqdm


In [5]:
# Load in the dotenv variables
load_dotenv()

# Grab the project path from the dotenv file without the addons
project_path = os.getenv('Project_Path')[2:78]

# Change notebook directory back one so that it can acess the data
os.chdir(project_path)

# Show all the columns in the .head() method
pd.set_option('display.max_columns', None)

In [7]:
# Load the cleaned data from data wrangling
data = pd.read_csv('./data/interim/wrangled', low_memory = False)
# Load in the loan_status column for use
loan = pd.read_csv('./data/raw/loan.csv',low_memory = False)
# Load the original dataset to get the accurate representation
y = loan['loan_status']